In [2]:
import redis
import pickle
from collections import defaultdict
import threading
import time
import numpy as np

In [35]:
class SpoofingNode:
    def __init__(self, node_id, fake_node_id, redis_host='localhost', redis_port=6379,):
        self.node_id = node_id
        self.fake_node_id = fake_node_id
        self.redis_client = redis.StrictRedis(host=redis_host, port=redis_port, db=0, password='secret_key')
        self.subscribe_channel1 = f'knn_euclidean_request_channel_{self.fake_node_id}'
        self.subscribe_channel2 = f'knn_cosine_request_channel_{self.fake_node_id}'
        self.subscribe_channel3 = f'knn_manhattan_request_channel_{self.fake_node_id}'
        self.publish_channel = 'knn_response_channel'
        self._listen_thread = threading.Thread(target=self.listen_for_requests)
        self._listen_thread.start()

    def listen_for_requests(self):
        pubsub = self.redis_client.pubsub()
        pubsub.subscribe(self.subscribe_channel1)
        pubsub.subscribe(self.subscribe_channel3)
        pubsub.subscribe(self.subscribe_channel2)
        print('hena')
        for message in pubsub.listen():
            
            try:
                if message['type'] == 'message':  
                    print(message['data'])            
                    request_data = pickle.loads(message['data'])
                    print(f'request data is {request_data}')
                    self.process_request(request_data
                    )
            except Exception:
                print("Can't unpickle data")
              
    def process_request(self,data):
        # Generate a fake prediction
        fake_prediction = [8 for _ in range(3)]  # Assuming 3 classes in the dataset
        fake_distances = [0 for _ in range(3)]
        response_data = {
            'node_id': self.fake_node_id,
            'prediction': fake_prediction,
            'distances': fake_distances,
            'request_id': data['request_id']
        }
        m = {'message': response_data, 'signature': "f"}
        self.redis_client.publish(self.publish_channel, pickle.dumps(m))


# Initialize the spoofing node
spoofing_node = SpoofingNode(node_id=4, fake_node_id=2)  # Pretending to be node 2


hena
b'\x80\x04\x95\x86\x02\x00\x00\x00\x00\x00\x00}\x94(\x8c\x06sample\x94]\x94(G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@\x1c\x00\x00\x00\x00\x00\x00G@(\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@\x10\x00\x00\x00\x00\x00\x00G@0\x00\x00\x00\x00\x00\x00G@ \x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@(\x00\x00\x00\x00\x00\x00G@&\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G\x00\x00\x00\x00\x00\x00\x00\x00G@.\x00\x00\x00\x00\x00\x00G@$\x00\x00\x00\x00\x00\x00G@ \x00\x00\x00\x00\x00\x00G@\x18\x00\x00\x